In [ ]:
# Only run this cell when in Google Colab
! git init
! git remote add origin https://github.com/hannamykula/oafbs.git
! git fetch
! git checkout -t origin/main

In [ ]:
! pip install tslearn
! pip install scikit-multiflow

In [33]:
from src.process import read_and_split_data, subsample_features
from src.model import train_candidates, cluster_predictions, compute_cluster_representatives, root_mean_square_error, plot_clustering, get_best_num_of_clusters, save_validation_predictions
from src.drift import PageHinkley, DataDrift
from src.predict import predict_n_steps_for_ensemble, predict_one_step_for_ensemble, get_weights, final_prediction_ensemble
import os
import pandas as pd
from config import EXPERIMENT_NAME, VALIDATION_WINDOW_SIZE, WEIGHTS_WINDOW_SIZE, EVALUATION_WINDOW, TARGET_INDEX, SUBSET_SIZE, K, MODEL

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
! wget -q --show-progress -P data/{EXPERIMENT_NAME}/ https://www.dropbox.com/s/ga07hldz2rizkuu/NEW-DATA-1.T15-PREPROCESSED.csv

In [34]:
train, val, test = read_and_split_data(EXPERIMENT_NAME + '/NEW-DATA-1.T15-PREPROCESSED.csv', val_size=VALIDATION_WINDOW_SIZE)

In [35]:
# Hyperparameters
target_index = TARGET_INDEX
subset_size = SUBSET_SIZE
k = K
model = MODEL

In [37]:
sample_subsets = subsample_features(train, target_index, subset_size, k)

In [69]:
train_candidates(train, val, target_index, sample_subsets, model)

Epoch 1/30
68/68 [==============================] - 3s 5ms/step - loss: 181.2070 - accuracy: 0.0000e+00
Epoch 2/30
68/68 [==============================] - 0s 6ms/step - loss: 106.5765 - accuracy: 0.0000e+00
Epoch 3/30
68/68 [==============================] - 0s 5ms/step - loss: 77.2542 - accuracy: 0.0000e+00
Epoch 4/30
68/68 [==============================] - 0s 5ms/step - loss: 63.6825 - accuracy: 0.0000e+00
Epoch 5/30
68/68 [==============================] - 0s 6ms/step - loss: 53.8405 - accuracy: 0.0000e+00
Epoch 6/30
68/68 [==============================] - 0s 5ms/step - loss: 46.1013 - accuracy: 0.0000e+00
Epoch 7/30
68/68 [==============================] - 0s 5ms/step - loss: 39.8710 - accuracy: 0.0000e+00
Epoch 8/30
68/68 [==============================] - 0s 6ms/step - loss: 34.8506 - accuracy: 0.0000e+00
Epoch 9/30
68/68 [==============================] - 0s 6ms/step - loss: 30.8152 - accuracy: 0.0000e+00
Epoch 10/30
68/68 [==============================] - 0s 5ms/step - loss

In [70]:
VALIDATION_FILENAME = os.path.join(os.getcwd(), 'experiments', EXPERIMENT_NAME, 'validation_predictions_init.csv')

In [71]:
validation_pred = pd.read_csv(VALIDATION_FILENAME)

In [72]:
# num_clusters = 30
num_clusters = get_best_num_of_clusters(validation_pred.transpose(), range(2,10))
cluster_result, cluster_centers = cluster_predictions(validation_pred.transpose(), num_clusters)
ensemble = compute_cluster_representatives(cluster_result, cluster_centers, validation_pred)

Determined number of clusters: 2.0


In [73]:
ph = PageHinkley(delta=0.005, threshold=0.025)
hoeffding = DataDrift(threshold=0.97)

In [74]:
all_model_indices = list(map(str, list(range(1, len(sample_subsets)+1))))

In [75]:
def compute_ensemble(X, running_version_name):
    val_predictions = predict_n_steps_for_ensemble(all_model_indices, X, sample_subsets)
    val_predictions = pd.DataFrame(val_predictions)
    val_predictions.columns = range(1, len(val_predictions.columns)+1)

    cluster_result, cluster_centers = cluster_predictions(val_predictions, num_clusters)
    transpose = val_predictions.transpose()
    transpose.columns = range(1, len(transpose.columns)+1)
    ensemble = compute_cluster_representatives(cluster_result, cluster_centers, transpose)

    return list(map(str, ensemble))

In [76]:
validation = pd.concat([val, test], axis=0)

In [78]:
start_index = 0
test_predictions = []
evaluation_sliding_window_start = 0
version = 1
while start_index < (len(test) - WEIGHTS_WINDOW_SIZE):
    end_index = start_index + WEIGHTS_WINDOW_SIZE
    X_window = test.iloc[start_index:end_index, ]
    y_window = test.iloc[start_index:end_index, target_index]
    X = test.iloc[end_index, ]
    print(start_index)
    weights = get_weights(ensemble, X_window, y_window, sample_subsets)
    pred_at_t = final_prediction_ensemble(ensemble, X, sample_subsets, weights)
    test_predictions.append(pred_at_t)
    if(len(test_predictions) >= EVALUATION_WINDOW):
        X = test.iloc[WEIGHTS_WINDOW_SIZE+evaluation_sliding_window_start:WEIGHTS_WINDOW_SIZE+evaluation_sliding_window_start+EVALUATION_WINDOW, test.columns!=test.columns[target_index]]
        y = test.iloc[WEIGHTS_WINDOW_SIZE+evaluation_sliding_window_start:WEIGHTS_WINDOW_SIZE+evaluation_sliding_window_start+EVALUATION_WINDOW, target_index]
        y_predicted = test_predictions[evaluation_sliding_window_start:evaluation_sliding_window_start+EVALUATION_WINDOW]
        error = root_mean_square_error(y.to_numpy(), y_predicted)
        print(f'Error for window [{WEIGHTS_WINDOW_SIZE+evaluation_sliding_window_start}:{WEIGHTS_WINDOW_SIZE+evaluation_sliding_window_start+EVALUATION_WINDOW}] is {error}')
        ph.add_element(error)
        print(f'Difference: {ph.sum - ph.minimum}')

        glob_min = hoeffding.min
        hoeffding.add_element(X, y)
        print(f'Hoeffding global minimum: {glob_min}. Latest minimum: {hoeffding.min2}. Sample count: {hoeffding.sample_count}')
        if ph.detected_change():
            print('Change in error detected.')
        if hoeffding.detected_change():
            print('Change in data detected')
        if ph.detected_change() | hoeffding.detected_change():
            X_val = validation.iloc[end_index:end_index + VALIDATION_WINDOW_SIZE, ]
            ensemble = compute_ensemble(X_val, version)
            version += 1
        evaluation_sliding_window_start += 1
    start_index = start_index + 1

0
1/1 [==============================] - 1s 505ms/step
1
1/1 [==============================] - 0s 495ms/step
2
1/1 [==============================] - 1s 513ms/step
3
1/1 [==============================] - 1s 503ms/step
4
1/1 [==============================] - 1s 531ms/step
5
1/1 [==============================] - 1s 506ms/step
6
1/1 [==============================] - 1s 542ms/step
7
1/1 [==============================] - 1s 879ms/step
8
1/1 [==============================] - 0s 496ms/step
9
1/1 [==============================] - 1s 678ms/step
10
1/1 [==============================] - 1s 560ms/step
11
1/1 [==============================] - 1s 545ms/step
12
1/1 [==============================] - 1s 540ms/step
13
1/1 [==============================] - 1s 567ms/step
14
1/1 [==============================] - 1s 535ms/step
15
1/1 [==============================] - 1s 551ms/step
16
1/1 [==============================] - 1s 581ms/step
17
1/1 [==============================] - 1s 509ms/step
18

In [79]:
root_mean_square_error(test.iloc[WEIGHTS_WINDOW_SIZE:, target_index].to_numpy(), test_predictions)

0.03574955954825183

In [80]:
from google.colab import drive
drive.mount('/content/drive')

save_name = EXPERIMENT_NAME + '_' + MODEL + '_k' + str(K) + '_results.csv'
path = F'/content/drive/MyDrive/Colab Notebooks/oafbs_results/{save_name}'
# path = os.path.join('results\sml10-dataset', save_name)

pd.DataFrame(test_predictions).to_csv(path)